In [0]:
# Imports
import numpy as np
import tensorflow as tf
from time import time
from tensorflow.nn import softmax_cross_entropy_with_logits_v2 as categorical_cross_entropy
from tensorflow.contrib.layers import conv2d, max_pool2d, fully_connected, flatten, dropout
#import tensorflow.nn.softmax
from tensorflow.losses import softmax_cross_entropy
from tensorflow.train import AdamOptimizer

In [15]:
data = np.load('test.npy')
data = data[()]
train_x = np.array(data['Data'])
train_y = np.array(data['Class_Value'])
print(train_x.shape)
print(train_y.shape)

samples = train_x
for i in range(len(train_x)):
   samples[i] = np.array(train_x[i])
   print(samples[i].shape)
print(samples[2])

(40,)
(40,)
(54, 76)
(60, 76)
(119, 76)
(72, 76)
(129, 76)
(84, 76)
(68, 76)
(81, 76)
(63, 76)
(95, 76)
(79, 76)
(63, 76)
(88, 76)
(39, 76)
(74, 76)
(108, 76)
(102, 76)
(90, 76)
(108, 76)
(60, 76)
(68, 76)
(30, 76)
(87, 76)
(150, 76)
(101, 76)
(110, 76)
(54, 76)
(71, 76)
(75, 76)
(68, 76)
(76, 76)
(83, 76)
(115, 76)
(99, 76)
(110, 76)
(97, 76)
(84, 76)
(54, 76)
(64, 76)
(95, 76)
[['0.15376700' '-0.05356000' '0.30664900' ... '-0.74965000' '-0.35224000'
  '-0.43486200']
 ['0.15392500' '-0.05260600' '0.30505000' ... '-0.86647100' '-0.62153000'
  '-0.44204700']
 ['0.15393900' '-0.05197800' '0.30414900' ... '-0.78757800' '-0.39792400'
  '-0.42882000']
 ...
 ['0.15983700' '-0.04261300' '0.29065300' ... '0.00144900' '-0.24994600'
  '-0.12020100']
 ['0.16114200' '-0.04068200' '0.28910400' ... '-0.06404700' '-0.52653600'
  '-0.10354500']
 ['0.16246700' '-0.03883900' '0.28764300' ... '-0.55719200' '-0.57501600'
  '-0.05733400']]


In [0]:
class Network(object):
  def __init__ (self, trainer):
    self.X = tf.placeholder(tf.float32, shape = [None, 60, 76]) # W x C where W is length and C is channels
    self.Y = tf.placeholder(tf.int64, shape= [None, 3])
    X_r = tf.reshape(self.X, [-1, 60, 76, 1]) 
    Y_r = tf.reshape(self.Y, [-1, 3])
    C1 = tf.contrib.layers.conv2d(inputs=X_r, num_outputs=38, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M1 = tf.contrib.layers.max_pool2d(inputs=C1, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D1 = tf.contrib.layers.dropout(inputs=M1, keep_prob=0.8)
    C2 = tf.contrib.layers.conv2d(inputs=D1, num_outputs=76, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M2 = tf.contrib.layers.max_pool2d(inputs=C2, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D2 = tf.contrib.layers.dropout(inputs=M2, keep_prob=0.8)
    C3 = tf.contrib.layers.conv2d(inputs=D2, num_outputs=152, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M3 = tf.contrib.layers.max_pool2d(inputs=C3, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D2 = tf.contrib.layers.dropout(inputs=M3, keep_prob=0.8)
    
    F1 = fully_connected(inputs=tf.contrib.layers.flatten(D2), num_outputs=64, activation_fn = tf.nn.softmax)
    FD1 = tf.contrib.layers.dropout(inputs=F1, keep_prob=0.5)
    F2 = fully_connected(inputs=FD1, num_outputs = 32)
    FD2 = tf.contrib.layers.dropout(inputs=F2, keep_prob=0.5)
    self.Out = fully_connected(inputs=FD2, num_outputs = 3, activation_fn = tf.nn.softmax)
    self.loss = tf.losses.softmax_cross_entropy(Y_r, self.Out)
    
    local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    self.gradients = tf.gradients(self.loss, local_vars)
    self.apply_grads = trainer.apply_gradients(zip(self.gradients, local_vars))
    
    self.accuracy =  tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(self.Out, 1), tf.argmax(self.Y, 1)), 
            tf.float32))
  
opt = AdamOptimizer()
net = Network(opt)


In [23]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())
  
    training_epochs = 50
    display_step = 1
    # Fit all training data
    for epoch in range(training_epochs):
      sess.run(net.apply_grads, feed_dict={net.X: train_X, net.Y: train_Y})

        # Display logs per epoch step
      if (epoch+1) % display_step == 0:
        c = sess.run(net.loss, feed_dict={net.X: train_X, net.Y: train_Y})
        acc = sess.run(net.accuracy, feed_dict={net.X: train_X, net.Y: train_Y})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{%f}" % c, "accuracy=", "{%f}" % acc) 
            # \"W=", sess.run(W), "b=", sess.run(b))

    print("Optimization Finished!")

NameError: ignored